In [402]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.models import Model



In [403]:
# Dummy dataset
data = {
    'Skintone': ['white', 'white', 'dark', 'medium', 'medium'],
    'r1': [239, 232, 240, 250, 245],
    'g1': [124, 180, 130, 140, 135],
    'b1': [142, 184, 150, 160, 155]
}



In [404]:
# Convert data to DataFrame
df = pd.DataFrame(data)



In [405]:
# Preprocess the data
df['Skintone'] = df['Skintone'].astype('category')
df['Skintone_code'] = df['Skintone'].cat.codes



In [406]:
# Define the embedding dimension
embedding_dim = 32



In [407]:
# Define the user embedding
user_embedding = Embedding(input_dim=df['Skintone_code'].nunique(), output_dim=embedding_dim)



In [408]:
# Define the model architecture
class RecommenderModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.user_embedding = user_embedding
        self.dense_layer = Dense(64, activation='relu')
        self.output_layer = Dense(3)  # Number of output features (r1, g1, b1)

    def call(self, inputs):
        user_embeddings = self.user_embedding(inputs[0])
        x = self.dense_layer(user_embeddings)
        output = self.output_layer(x)
        return output

    def compute_loss(self, features, training=False):
        target = features[1]
        predicted_output = self(features)
        loss = tf.reduce_mean(tf.square(target - predicted_output))
        return loss



In [409]:
model = RecommenderModel()



In [410]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())



In [411]:
# Create the TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(({'Skintone_code': df['Skintone_code']}, df[['r1', 'g1', 'b1']]))
dataset = dataset.map(lambda x, y: (x, {'target': y})).batch(32)


In [412]:

# Train the model
model.fit(dataset, epochs=10)



Epoch 1/10


AttributeError: in user code:

    File "d:\Github\Capstone-ChromaMatch\.venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "d:\Github\Capstone-ChromaMatch\.venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Github\Capstone-ChromaMatch\.venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "d:\Github\Capstone-ChromaMatch\.venv\lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\drago\AppData\Local\Temp\ipykernel_11296\2949879.py", line 18, in compute_loss
        predicted_output = self(features)
    File "d:\Github\Capstone-ChromaMatch\.venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_han

In [ ]:
# Example usage: Predict values for a given Skintone
skintone_code = 2  # Replace with the desired Skintone code
prediction = model({'Skintone_code': tf.constant([skintone_code])}).numpy()
print(f"Predicted values for Skintone {df['Skintone'].cat.categories[skintone_code]}:")
print(prediction[0])